In [1]:
import pylab as plt
%matplotlib inline
import numpy as np
import pandas as pd
import math

from featureEngineering2 import feature_engineering

df=pd.read_csv('data/train.csv', sep=',')

df_f = feature_engineering(df)

df_d = feature_engineering(df, get_dummies=True)



Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days


In [65]:
from sklearn.ensemble import RandomForestClassifier

feat_labels = df_d.columns
X_train = df_d
y_train = df['Survived']


In [66]:
from sklearn.cross_validation import train_test_split
X, y = df_d.iloc[:].values, df['Survived'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

######################################

from sklearn.pipeline import Pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

select = SelectFromModel(forest, prefit=True, threshold=0.00)
X_train = select.transform(X_train)
X_test = select.transform(X_test)

pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', KernelPCA(n_components=8, kernel='rbf', gamma=0.05)),
        #('clf', LogisticRegression(random_state=1))
        ('clf', KNeighborsClassifier(n_neighbors=5))
        #('clf', RandomForestClassifier(n_estimators=100,
                                      # criterion='entropy',
                                       #random_state=0,
                                       #max_depth=4,
                                       ##max_features=4,
                                       #n_jobs=-1))
    ])

pipe.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', KernelPCA(alpha=1.0, coef0=1, degree=3, eigen_solver='auto',
     fit_inverse_transform=False, gamma=0.05, kernel='rbf',
     kernel_params=None, max_iter=None, n_components=8,
     remove_zero_eig=False, tol=0)), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])

In [67]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

print accuracy_score(pipe.predict(X_test), y_test)
print accuracy_score(pipe.predict(X_train), y_train)
print precision_score(pipe.predict(X_test), y_test)
print f1_score(pipe.predict(X_test), y_test)

0.776536312849
0.862359550562
0.657534246575
0.705882352941


In [64]:
from sklearn import svm, cross_validation, datasets

X, y = df_d.iloc[:].values, df['Survived'].values
select = SelectFromModel(forest, prefit=True, threshold=0.00)
X = select.transform(X)

scores = cross_validation.cross_val_score(pipe, X, y, scoring='f1', cv=8)

print('F1: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

F1: 0.741 +/- 0.048


## Tuning hyperparameters via grid search

In [75]:
from sklearn.grid_search import GridSearchCV

pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', KernelPCA(kernel='rbf')),
        #('clf', LogisticRegression(random_state=1))
        ('clf', KNeighborsClassifier())
        #('clf', RandomForestClassifier(n_estimators=100,
                                      # criterion='entropy',
                                       #random_state=0,
                                       #max_depth=4,
                                       ##max_features=4,
                                       #n_jobs=-1))
    ])

pca_gamma_range = np.linspace(0.01, 0.2, 10)
pca_n_range = range(4,10)
knn_n_neighbor_range = range(4,10)

param_grid = [
    {'pca__n_components': pca_n_range,
     'pca__gamma': pca_gamma_range,
     'clf__n_neighbors': knn_n_neighbor_range
    }
]

gs = GridSearchCV(estimator=pipe,
                  param_grid=param_grid,
                  scoring='f1',
                  cv=8,
                  n_jobs=-1)

gs = gs.fit(X, y)

print gs.best_score_

print gs.best_params_




0.76112972569
{'pca__n_components': 7, 'pca__gamma': 0.03111111111111111, 'clf__n_neighbors': 7}


In [74]:
X.shape

(891, 38)

In [76]:
scores = cross_validation.cross_val_score(gs, X, y, scoring='f1', cv=5)
print('F1: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

F1: 0.712 +/- 0.052


In [77]:
scores

array([ 0.69491525,  0.71532847,  0.72307692,  0.63157895,  0.79411765])